In [29]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import boxscoresummaryv3
import sys
from pathlib import Path
import pandas as pd


# Get the parent directory (Datos folder)
current_dir = Path().resolve()  # libretas folder
datos_dir = current_dir.parent  # Datos folder

# Add Datos to path so we can import from Functions
sys.path.insert(0, str(datos_dir))



pd.set_option("display.max_columns", None)


In [30]:

from Functions.helpfuncs import get_current_season
import numpy as np
from typing import List, Dict, Tuple, Union, Optional

In [52]:
class Games():
    def __init__(self):
        self.current_season = get_current_season()

    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_team_game_log(self, team_id: int, season: str) -> pd.DataFrame:
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log

    def get_todays_games(self) -> None:
        f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

    def get_game_by_id(self, game_id: str) -> Dict:
        """Get game details by game ID"""
        try:
            
            game_summary = boxscoresummaryv3.BoxScoreSummaryV3(game_id=game_id)
            
            # Get all dataframes
            game_summary_data = game_summary.get_data_frames()[7]
            line_score = game_summary.get_data_frames()[1]
            
            return game_summary_data
            
        except Exception as e:
            print(f"Error retrieving game {game_id}: {e}")
            raise e

    def get_current_standings(self, season: str = None, conference:str = 'Overall') -> pd.DataFrame:
        """Get current NBA standings for all teams"""
        conference_types = ['Overall','West','East']
        try:
            if conference in conference_types:
                season = self.check_valid_season(season)
                
                standings = leaguestandingsv3.LeagueStandingsV3(
                    league_id='00',  # NBA
                    season=season,
                    season_type='Regular Season'
                )
                
                standings_df = standings.get_data_frames()[0]
                
                # Select relevant columns
                columns_to_keep = [
                    'TeamID', 'TeamCity', 'TeamName', 'Conference','WINS', 'LOSSES', 'PlayoffRank', 
                    'Record', "HOME","ROAD", "L10"
                ]
                
                standings_df = standings_df[columns_to_keep]

                if conference != 'Overall':
                    return standings_df[standings_df['Conference'] == conference]
                else:
                    return standings_df
            else:
                raise ValueError(f"Invalid conference type: {conference}. Expected one of {conference_types}.")    
        except Exception as e:
            print(f"Error retrieving standings for season {season}: {e}")
            raise e


In [53]:
Games().get_game_by_id(game_id='0022500278')

,gameId,teamId,teamCity,teamName,teamTricode,points,reboundsTotal,assists,steals,blocks,turnovers,fieldGoalsPercentage,threePointersPercentage,freeThrowsPercentage,pointsInThePaint,pointsSecondChance,pointsFastBreak,biggestLead,leadChanges,timesTied,biggestScoringRun,turnoversTeam,turnoversTotal,reboundsTeam,pointsFromTurnovers,benchPoints
0,0022500278,1610612739,Cleveland,Cavaliers,CLE,120.0,41.0,28.0,10.0,4.0,12.0,0.517,0.378,0.650,40.0,21.0,14.0,20.0,1.0,1.0,12.0,2.0,14.0,10.0,19.0,33.0
1,0022500278,1610612746,LA,Clippers,LAC,105.0,40.0,23.0,6.0,2.0,15.0,0.463,0.308,0.885,52.0,12.0,9.0,8.0,1.0,1.0,11.0,1.0,16.0,4.0,22.0,19.0


In [9]:
Games().get_todays_games()

ScoreBoardDate: 2025-11-28
0022500072: Magic @ Pistons : 2025-11-28 17:30:00-07:00
0022500073: Wizards @ Pacers : 2025-11-28 17:30:00-07:00
0022500074: Bucks @ Knicks : 2025-11-28 17:30:00-07:00
0022500069: Cavaliers @ Hawks : 2025-11-28 17:30:00-07:00
0022500071: Bulls @ Hornets : 2025-11-28 17:30:00-07:00
0022500070: 76ers @ Nets : 2025-11-28 17:30:00-07:00
0022500075: Suns @ Thunder : 2025-11-28 19:30:00-07:00
0022500076: Spurs @ Nuggets : 2025-11-28 19:30:00-07:00
0022500077: Kings @ Jazz : 2025-11-28 19:30:00-07:00
0022500078: Mavericks @ Lakers : 2025-11-28 20:00:00-07:00
0022500079: Grizzlies @ Clippers : 2025-11-28 20:00:00-07:00
